# Dog Breed Recognition Project

## Project Basics

#### Problem

Our goal is to identify dog breed from a photo of the dog.  
The project is taken from [Kaggle Dog Breed Identification Competition](https://www.kaggle.com/c/dog-breed-identification/data).  
The machine learning problem is **supervised learning > multiclass classification**.  
Our task is to build a neural network image classifier using TensorFlow and TensorFlow Hub.

#### Evaluation

The evaluation metric set for the competition is Multiclass Log Loss.  
Our target matrix contains N Dogs x M Breeds, true breed = 1, rest = 0.  
Our model predicts a probability matrix with the same dimensions.  
Multiclass Log Loss measures the error of model predictions (the lower the better).  
Muticlass Log Loss is applied in image classification, natural language processing, and recommendation systems.

#### Data Source

Data is acquired from [Kaggle Dog Breed Identification Competition](https://www.kaggle.com/c/dog-breed-identification/data).

#### Features / Data Dictionary

Our model analyzes image files (unstructured data) > deep learning / transfer learning.  
There are 120 unique dog breeds in the training set > multiclass classification with 120 classes.  
There are 10 222 images in the training set.  
There are 10 357 images in the test set.

## Preparing Tools

In [ ]:
### importing tensorflow
import tensorflow as tflow
print(tflow.__version__)
print(tflow.config.list_physical_devices())

### importing tensorflow hub
import tensorflow_hub as thub
print(thub.__version__)

### python libraries
from typing import Tuple
from pathlib import Path
from datetime import datetime

### external libraries
import numpy
from pandas import read_csv, DataFrame, concat
from matplotlib import pyplot
from IPython.display import Image

In [ ]:
### global variable declarations
AUTOTUNE = tflow.data.AUTOTUNE

## Preparing Data

#### Creating Train Image/Label Dataframe

In [ ]:
### extracting list of filepaths from train folder
train_files = [file for file in Path("D:/WorkDev_GitHub/projectData/dataDogRecognition/train").iterdir()]
len(train_files)

In [ ]:
### importing true labels for train images
train_df = read_csv(filepath_or_buffer="data-train-labels.csv")

In [ ]:
### creating train imagepaths from image ids
train_df["imagepath"] = "D:/WorkDev_GitHub/projectData/dataDogRecognition/train/" + train_df["id"] + ".jpg"
train_df = train_df.loc[:, ["imagepath","breed"]].copy(deep=True)

In [ ]:
### creating unique breeds array
unique_breeds = train_df["breed"].to_numpy()
unique_breeds = numpy.unique(ar=unique_breeds)
unique_breeds.shape

In [ ]:
### computing mean of images/breed
round(number=train_df["breed"].value_counts().mean(), ndigits=3)

Google recommends at least 10 images per class.  
We have adequate data with ~85 images per class on average.

In [ ]:
### numerical encoding of dog breeds
train_df["label"] = train_df["breed"].map(arg=lambda item: numpy.where(unique_breeds == item)[0][0])
train_df = train_df.loc[:, ["imagepath","label"]].copy(deep=True)

In [ ]:
### exploring train dataframe: head
train_df.head()

In [ ]:
### exploring train dataframe: info
train_df.info()

In [ ]:
### exploring train dataframe: checking validity of random imagepath
print(unique_breeds[train_df.loc[9000, "label"]])
print()
print(train_df.loc[9000, "imagepath"])
print()
Image(filename=train_df.loc[9000, "imagepath"])

#### Creating Test Image Dataframe

In [ ]:
### creating list of filepaths in test folder
test_files = [str(file) for file in Path("D:/WorkDev_GitHub/projectData/dataDogRecognition/test").iterdir()]
len(test_files)

In [ ]:
### creating test dataframe
test_df = DataFrame(data=test_files, columns=["imagepath"])

In [ ]:
### exploring test dataframe: head
test_df.head()

In [ ]:
### exploring test dataframe: info
test_df.info()

In [ ]:
### exploring test datraframe: checking validity of random imagepath
print(test_df.loc[855, "imagepath"])
print()
Image(filename=test_df.loc[855, "imagepath"])

#### Reducing and Splitting

In [ ]:
### dataframe inits
rTrain_df = DataFrame()
rValid_df = DataFrame()
fTrain_df = DataFrame()
fValid_df = DataFrame()

In [ ]:
### creating train and valid dataframes
for label in range(120):
    work_df: DataFrame = DataFrame()
    work_df = train_df.loc[train_df["label"] == label]
    work_df = work_df.sample(n=work_df.index.size, random_state=42)
    train_num = int(0.9 * work_df.index.size)
    valid_num = work_df.index.size - train_num
    fTrain_df = concat(objs=[fTrain_df, work_df.iloc[:train_num]])
    fValid_df = concat(objs=[fValid_df, work_df.iloc[train_num:]])
    work_df = work_df.sample(n=12, random_state=42)
    rTrain_df = concat(objs=[rTrain_df, work_df.iloc[:10]])
    rValid_df = concat(objs=[rValid_df, work_df.iloc[10:]])

In [ ]:
### shuffling train dataframes
rTrain_df = rTrain_df.sample(n=rTrain_df.index.size, random_state=42)
fTrain_df = fTrain_df.sample(n=fTrain_df.index.size, random_state=42)

In [ ]:
### verifying dimensions of dataframes
rTrain_df.shape, rValid_df.shape, fTrain_df.shape, fValid_df.shape, test_df.shape

#### Creating Datasets

All machine learning algorithms require data in numerical format.  
So the first task is to turn images and labels into tensors.  
A tensor is a numerical matrix with n-dimensions, like a numpy ndarray.

In [ ]:
### function creating image tensor
def tensorImage(aImage_path=tflow.Tensor(), aImage_size=224):
    """
    Converts image file into constant image tensor.
    """
    image_tensor = tflow.io.read_file(filename=aImage_path)
    image_tensor = tflow.image.decode_jpeg(contents=image_tensor, channels=3)
    image_tensor = tflow.image.convert_image_dtype(image=image_tensor, dtype=tflow.float32)
    image_tensor = tflow.image.resize(images=image_tensor, size=[aImage_size,aImage_size])
    return image_tensor

In [ ]:
### function creating image/label tensor
def tensorImageLabel(tImage_path=tflow.Tensor(), aLabel=tflow.Tensor()):
    """
    Invokes tensorImage() function.\n
    One Hot Encodes true dog breed label into constant label tensor.
    """
    image_tensor = tensorImage(aImage_path=tImage_path, aImage_size=224)
    encoder = tflow.keras.layers.CategoryEncoding(num_tokens=120, output_mode="one_hot")
    label_tensor = encoder(aLabel)
    label_tensor = tflow.cast(x=label_tensor, dtype=tflow.int8)
    return image_tensor, label_tensor

In [ ]:
### function creating dataset
def createDataset(aInput_df=DataFrame(), aTest=False):
    """
    Creates keras dataset.
    """
    tensor_imagepath = aInput_df["imagepath"].to_numpy()
    if aTest:
        dataset = tflow.data.Dataset.from_tensor_slices(tensors=tensor_imagepath)
        dataset = dataset.map(map_func=tensorImage, num_parallel_calls=AUTOTUNE, deterministic=True)
    else:
        tensor_label = aInput_df["label"].to_numpy()
        dataset = tflow.data.Dataset.from_tensor_slices(tensors=(tensor_imagepath,tensor_label))
        dataset = dataset.map(map_func=tensorImageLabel, num_parallel_calls=AUTOTUNE, deterministic=True)
    return dataset

In [ ]:
### creating reduced train dataset
rTrain_ds = createDataset(aInput_df=rTrain_df, aTest=False)
rTrain_ds.element_spec

In [ ]:
### creating reduced valid dataset
rValid_ds = createDataset(aInput_df=rValid_df, aTest=False)
rValid_ds.element_spec

In [ ]:
### creating full train dataset
fTrain_ds = createDataset(aInput_df=fTrain_df, aTest=False)
fTrain_ds.element_spec

In [ ]:
### creating full valid dataset
fValid_ds = createDataset(aInput_df=fValid_df, aTest=False)
fValid_ds.element_spec

In [ ]:
### creating test dataset
test_ds = createDataset(aInput_df=test_df, aTest=True)
test_ds.element_spec

#### Visualizing Datasets

In [ ]:
### function visualizing first 32 tensors
def visualizeTensors(
        aDataset=tflow.data.Dataset.from_tensor_slices(tensors=(numpy.array([]),numpy.array([]))),
        aTest=False):
    """
    Plots the first 32 tensors from the given dataset.
    """
    index = 1
    pyplot.figure(figsize=(8,12))
    for tensors in aDataset.as_numpy_iterator():
        pyplot.subplot(8, 4, index)
        if aTest:
            tensors: numpy.ndarray
            pyplot.imshow(X=tensors)
        else:
            tensors: Tuple[numpy.ndarray,numpy.ndarray]
            pyplot.imshow(X=tensors[0])
            pyplot.title(label=unique_breeds[tensors[1].argmax()], fontsize=8)
        pyplot.axis("off")
        index += 1
        if 32 < index: break
    return

In [ ]:
### visualizing reduced train dataset
visualizeTensors(aDataset=rTrain_ds, aTest=False)

In [ ]:
### visualizing reduced valid dataset
visualizeTensors(aDataset=rValid_ds, aTest=False)

In [ ]:
### visualizing full train dataset
visualizeTensors(aDataset=fTrain_ds, aTest=False)

In [ ]:
### visualizing full valid dataset
visualizeTensors(aDataset=fValid_ds, aTest=False)

In [ ]:
### visualizing test dataset
visualizeTensors(aDataset=test_ds, aTest=True)

## Preparing Neural Network

#### Defining Network Components

**Optimal network parameters:**  
Binary classification: sigmoid (activation), binary crossentropy (loss)  
Multiclass classification: softmax (activation), categorical crossentropy (loss)

In [ ]:
### function defining neural network architecture
def buildNetwork(aInput_shape=list(), aModel_url=str(), aOutput_shape=int()):
    """
    Builds a deep learning neural network.
    """
    ### defining network architecture
    neural_net = tflow.keras.Sequential([
        thub.KerasLayer(handle=aModel_url), # input layer
        tflow.keras.layers.Dense(units=aOutput_shape, activation="softmax")]) # output layer
    ### setting metrics and optimization
    neural_net.compile(
        loss=tflow.keras.losses.CategoricalCrossentropy(), # error function
        optimizer=tflow.keras.optimizers.Adam(), # optimizer algorithm
        metrics=["accuracy"])
    ### building and returning network
    neural_net.build(input_shape=aInput_shape)
    return neural_net

**Callbacks are event handler functions that are called at certain model training events.**  
The TensorBoard() callback saves a training log that helps in monitoring the training process.  
The EarlyStopping() callback halts training when a chosen metric stops improving.  

In [ ]:
### function creating tensorboard callback
def callbackTensorboard():
    """
    Creates folder structure for TensorBoard logs.\
    Returns TensorBoard callback object.
    """
    log_root = Path("./logs")
    log_folder = log_root.joinpath(datetime.now().strftime("%Y%m%d-%H%M%S"))
    return tflow.keras.callbacks.TensorBoard(log_dir=log_folder)

In [ ]:
### function creating early stopping callback
def callbackEarlyStopping():
    """
    Creates and returns an EarlyStopping callback object.
    """
    return tflow.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

#### Training and Saving Model

GPUs have limited amount of memory.  
The entire training dataset may not fit into GPU memory.  
To resolve this, we split our datasets into batches of ~32 tensors.  
The neural network sees only one batch at a time.

In [ ]:
### selectinig and configuring train dataset for performance
train_ds = rTrain_ds.cache()
train_ds = train_ds.batch(batch_size=32, num_parallel_calls=AUTOTUNE, deterministic=True)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
### selectinig and configuring valid dataset for performance
valid_ds = rValid_ds.cache()
valid_ds = valid_ds.batch(batch_size=32, num_parallel_calls=AUTOTUNE, deterministic=True)
valid_ds = valid_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
### training and saving neural network
recognition_model = buildNetwork(
    aInput_shape=[None, 224, 224, 3],
    aModel_url="https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/130-224-classification/2", aOutput_shape=len(unique_breeds))
recognition_model.fit(
    x=train_ds,
    validation_data=valid_ds,
    epochs=100,
    validation_freq=1,
    callbacks=[callbackTensorboard(),callbackEarlyStopping()])
recognition_model.save(filepath="./models/testModel")

#### Model Evaluation with TensorBoard

In [ ]:
### loading tensorboard
#> launch terminal
#> run command: trensorboard --logdir ./logs

#### Loading Trained Model

In [ ]:
### loading and evaluating trained model
recognition_loaded: tflow.keras.Model = tflow.keras.models.load_model(
    filepath="./models/testModel",
    custom_objects={"KerasLayer":thub.KerasLayer})
recognition_loaded.evaluate(valid_ds)

## Processing Predictions

#### Making Predictions

In [ ]:
### making predictions
valid_preds: numpy.ndarray = recognition_loaded.predict(x=valid_ds, verbose=True)
valid_preds.shape

#### Visualizing Predictions

In [ ]:
### function plotting image, true label, and top predictions
def plotPredictions(aImage=numpy.array([]), aLabel=numpy.array([]), aPred=numpy.array([])):
    """
    Plots an image of a dog, its true breed, and the top predicted breeds.
    """
    ### figure init
    pyplot.figure(figsize=(7,4.5))
    ### plotting image and true label
    image_plot = pyplot.subplot(1, 2, 1)
    image_plot.imshow(X=aImage)
    image_plot.set_title(label=unique_breeds[numpy.argmax(a=aLabel)], fontsize=10)
    image_plot.set_axis_off()
    # image_plot.set_yticks(ticks=[])
    # image_plot.set_xticks(ticks=[])
    ### plotting top 5 predictions
    top_indexes = numpy.argsort(a=aPred)[-5:][::-1]
    pred_plot = pyplot.subplot(1, 2, 2)
    pred_plot.bar(height=aPred[top_indexes], x=range(5))
    pred_plot.set_title(label="Top Five Predictions", fontsize=10)
    pred_plot.set_ylabel(ylabel="Confidence Levels", fontsize=10)
    pred_plot.set_xticks(ticks=range(5), labels=unique_breeds[top_indexes], rotation="vertical")
    ### layout and returning
    pyplot.tight_layout(h_pad=0.1)
    return

In [ ]:
### visualizing select predictions
INDEX = 239
for index,item in enumerate(iterable=rValid_ds.as_numpy_iterator()):
    if index == INDEX:
        plotPredictions(aImage=item[0], aLabel=item[1], aPred=valid_preds[INDEX])

Challenge: Create a confusion matrix of true labels versus predictions.

## Applying Model on Test Images

In [ ]:
### configuring test dataset for performance
test_ds = test_ds.cache()
test_ds = test_ds.batch(batch_size=32, num_parallel_calls=AUTOTUNE, deterministic=True)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
### making predictions on test images
test_preds: numpy.ndarray = recognition_loaded.predict(x=test_ds, verbose=True)
test_preds.shape

In [ ]:
### saving test predictions
numpy.savetxt(X=test_preds, delimiter=",", fname="./data-test-preds.csv")

In [ ]:
### loading test predictions
preds_loaded = numpy.loadtxt(fname="./data-test-preds.csv", delimiter=",")
preds_loaded.shape